In [146]:
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np


In [147]:
#꼬꼬마분석기
kkma = Kkma() 

#text를 입력받아 Kkma.sentences()를 이용해 문장단위로 나눈 뒤 sentences로 리턴
def text2sentences(text):  
    sentences = kkma.sentences(text)  
    #text일 때 문장별로 리스트 만듦
    return sentences

In [148]:
#예시 줄거리
desc='''아이스댄싱이라는 흔하지 않은 종목에서 완벽한 파트너를 만나는 일은 기적과 같은 일. 파트너 운이 없지만 꾸준히 아이스댄싱에 대한 남다른 애정을 가지고 있는 홍주와 과거 부상에서 다시 아이스댄싱을 시작하는 해성이의 기적같은 만남이 빙판을 금빛으로 물들인다.'''

In [149]:
sentences =text2sentences(desc)
print(sentences)

['아이스댄싱이라는 흔하지 않은 종목에서 완벽한 파트너를 만나는 일은 기적과 같은 일. 파트너 운이 없지만 꾸준히 아이스댄싱에 대한 남다른 애정을 가지고 있는 홍 주와 과거 부상에서 다시 아이스댄싱을 시작하는 해성 이의 기적 같은 만남이 빙판을 금빛으로 물들인다.']


In [150]:
twitter = Twitter()

#불용어제거 수정하기
stopwords = ['네이버','네이버웹툰', '최강자전', '대표작', '신작',
             '그', '것', '누구',' 자신',
            '를', '을', '나이', '매일']
def get_nouns(sentences):
    nouns = []
    for sentence in sentences:
        if sentence is not '':
            nouns.append(' '.join([noun for noun in twitter.nouns(str(sentence))
                                  if noun not in stopwords and len(noun) >= 1]))
    return nouns

<>:10: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:10: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
C:\Users\user\AppData\Local\Temp/ipykernel_36880/1320837504.py:10: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if sentence is not '':


In [151]:
nouns = get_nouns(sentences)

In [152]:

nouns = get_nouns(sentences)
print(sentences[0])
print(nouns[0])


아이스댄싱이라는 흔하지 않은 종목에서 완벽한 파트너를 만나는 일은 기적과 같은 일. 파트너 운이 없지만 꾸준히 아이스댄싱에 대한 남다른 애정을 가지고 있는 홍 주와 과거 부상에서 다시 아이스댄싱을 시작하는 해성 이의 기적 같은 만남이 빙판을 금빛으로 물들인다.
아이스댄싱 종목 파트너 일 기적 일 파트너 운 아이스댄싱 대한 애정 가지 홍 주 과거 부상 다시 아이스댄싱 시작 해성 이의 기적 만남 빙판 금빛


In [153]:
tfidf = TfidfVectorizer()
cnt_vec = CountVectorizer()
graph_sentence = []

In [154]:
def build_words_graph(sentence):
    cnt_vec_mat = normalize(cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
    vocab = cnt_vec.vocabulary_
    return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}
    
words_graph, idx2word = build_words_graph(nouns)
print(idx2word)

{10: '아이스댄싱', 13: '종목', 14: '파트너', 3: '기적', 5: '대한', 11: '애정', 0: '가지', 1: '과거', 7: '부상', 4: '다시', 9: '시작', 15: '해성', 12: '이의', 6: '만남', 8: '빙판', 2: '금빛'}


In [155]:
def get_ranks(graph, d=0.85): 
    A = graph
    matrix_size = A.shape[0]
    for id in range(matrix_size):
        A[id, id] = 0 
        link_sum = np.sum(A[:,id]) 
        if link_sum != 0:
            A[:, id] /= link_sum
        A[:, id] *= -d
        A[id, id] = 1

    B = (1-d) * np.ones((matrix_size, 1))
    ranks = np.linalg.solve(A, B) 
    return {idx: r[0] for idx, r in enumerate(ranks)}

In [156]:
word_rank_idx = get_ranks(words_graph)
print(word_rank_idx)

{0: 1.0000000000000002, 1: 1.0000000000000004, 2: 1.0, 3: 1.0, 4: 1.0000000000000007, 5: 1.0000000000000007, 6: 1.0000000000000002, 7: 1.0000000000000002, 8: 1.0000000000000002, 9: 1.0000000000000004, 10: 1.0000000000000002, 11: 1.0000000000000002, 12: 1.0000000000000002, 13: 1.0000000000000002, 14: 1.0000000000000002, 15: 1.0000000000000002}


In [157]:
sorted_word_rank_idx = sorted(word_rank_idx, key=lambda k: word_rank_idx[k], reverse=True)

In [158]:
# 키워드 수 조정
def keywords(word_num=10):

    keywords = []
    index=[]
    for idx in sorted_word_rank_idx[:word_num]:
        index.append(idx)

    #index.sort()
    for idx in index:
        keywords.append(idx2word[idx])

    print(keywords)

In [159]:
keywords()

['다시', '대한', '과거', '시작', '가지', '만남', '부상', '빙판', '아이스댄싱', '애정']
